# kNN & Distance-Based Learning — Student Lab

Complete all TODOs. Focus: vectorization, scaling, and failure modes.

In [1]:
# KNN used in both for classification and regression
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic dataset generator
We’ll generate Gaussian blobs with controllable dimension.

In [ ]:
# Guassian blob = this is clouds of points drawn from Gaussian distributions
def make_blobs(n_train=400, n_test=200, d=2, sep=2.5):
    # two Gaussian blobs
    mu0 = np.zeros(d)# mu0 = [0,0,0,0,0] , d = 5
    mu1 = np.zeros(d); mu1[0] = sep # mu1 = [sep,0,0,0,0]
    X0 = rng.standard_normal((n_train//2, d)) + mu0
    X1 = rng.standard_normal((n_train - n_train//2, d)) + mu1
    X_train = np.vstack([X0, X1])
    y_train = np.array([0]*len(X0) + [1]*len(X1))
    perm = rng.permutation(n_train)
    X_train, y_train = X_train[perm], y_train[perm]

    T0 = rng.standard_normal((n_test//2, d)) + mu0
    T1 = rng.standard_normal((n_test - n_test//2, d)) + mu1
    X_test = np.vstack([T0, T1])
    y_test = np.array([0]*len(T0) + [1]*len(T1))
    perm = rng.permutation(n_test)
    X_test, y_test = X_test[perm], y_test[perm]
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = make_blobs(d=5)
check('shapes', X_train.shape[0] == y_train.shape[0] and X_test.shape[0] == y_test.shape[0])
X_train.shape, X_test.shape

OK: shapes


((400, 5), (200, 5))

## Section 1 — Vectorized distances

### Task 1.1: L2 distance matrix
Compute D where D[i,j] = ||X_test[i] - X_train[j]||_2.

# HINT:
- Use expansion: ||a-b||^2 = ||a||^2 + ||b||^2 - 2 a·b
- Avoid building (m,n,d) broadcast tensor for large sizes


In [ ]:
# vectorization is faster than loops
# computing distances of many points at once.
# l2 distance = Euclidean distance
def l2_distances(X_test, X_train):
    # TODO: return (m,n) matrix
    a2 = np.sum(X_test * X_test, axis=1, keepdims=True)  # (m,1) 
    b2 = np.sum(X_train * X_train, axis=1, keepdims=True).T # (1,n)
    dist2 = a2 + b2 - 2*(X_test @ X_train.T ) # (m,n)
    dist2 = np.maximum(dist2, 0.0)  # numerical stability
    return np.sqrt(dist2)

D = l2_distances(X_test[:10], X_train[:20])
check('D_shape', D.shape == (10, 20))
check('D_nonneg', np.all(D >= -1e-9))

OK: D_shape
OK: D_nonneg


### Task 1.2: L1 distances (optional)
Compute L1 distance matrix (can use broadcasting here since we’ll keep sizes small).

**Interview Angle:** When might L1 beat L2?

In [ ]:
# l1 is Manhattan distance more robust to outliers than l2
def l1_distances(X_test, X_train):
    # TODO
    return np.sum(np.abs(X_test[:, None, :] - X_train[None, :, :]), axis=2)

D1 = l1_distances(X_test[:5], X_train[:7])
check('D1_shape', D1.shape == (5, 7))

OK: D1_shape


## Section 2 — kNN classifier

### Task 2.1: Predict with kNN

# HINT:
- compute distances
- argsort distances to find k nearest
- majority vote

**FAANG gotcha:** define tie-breaking deterministically (e.g., pick smallest label).

In [10]:
def knn_predict(X_train, y_train, X_test, k=3):
    # TODO
    D = l2_distances(X_test, X_train)  # (m,n)
    nn_idx = np.argsort(D, axis=1)[:, :k]  # (m,k) # np.argsort returns indices that would sort an array row-wise
    # print(nn_idx.shape)
    nn_labels = y_train[nn_idx] 
    # print(nn_labels.shape)
    # majority vote
    votes = nn_labels.sum(axis=1) # here votes are assuming binary classification 0 and 1 
    return (votes * 2 > k).astype(int) # if votes > k/2 then class 1(Apple) else class 0(Grapes)

yhat = knn_predict(X_train, y_train, X_test[:20], k=3)
check('yhat_shape', yhat.shape == (20,))
check('labels', set(np.unique(yhat)).issubset({0,1}))

OK: yhat_shape
OK: labels


### Task 2.2: Evaluate over k
Compute accuracy for k in [1,3,5,9,15].

**Checkpoint:** Why does increasing k usually increase bias and reduce variance?

In [11]:
def accuracy(y, yhat):
    return float(np.mean(y == yhat))

ks = [1,3,5,9,15]
for k in ks:
    yhat_tr = knn_predict(X_train, y_train, X_train, k=k)
    yhat_te = knn_predict(X_train, y_train, X_test, k=k)
    print('k', k, 'train_acc', accuracy(y_train, yhat_tr), 'test_acc', accuracy(y_test, yhat_te))

k 1 train_acc 1.0 test_acc 0.825
k 3 train_acc 0.9325 test_acc 0.87
k 5 train_acc 0.905 test_acc 0.885
k 9 train_acc 0.89 test_acc 0.88
k 15 train_acc 0.885 test_acc 0.89


## Section 3 — Curse of dimensionality

### Task 3.1: Distance concentration
For increasing dimension d, compute ratio min_dist/max_dist for random points and show it approaches 1.

# HINT:
- sample n points in d dims
- compute pairwise distances from one reference point


In [ ]:
# demonstrate concentration of distances in high dimensions
# distance concentration means that in high-dimensional spaces, the distances between points tend to become similar, making it difficult to distinguish between them based on distance metrics.
def concentration_ratio(n=2000, dims=(2,5,10,50,100)):
    ratios = []
    for d in dims:
        X = rng.standard_normal((n, d))
        ref = X[0:1] 
        D = l2_distances(ref, X)[0, 1:]
        ratios.append((d, float(D.min() / D.max())))
    return ratios

ratios = concentration_ratio()
for d, r in ratios:
    print('d', d, 'min/max', r)

d 2 min/max 0.006214278718774362
d 5 min/max 0.10968424369044452
d 10 min/max 0.28327247598849514
d 50 min/max 0.4947050997185768
d 100 min/max 0.6569330655910577


## Section 4 — Feature scaling sensitivity

### Task 4.1: Break kNN with scaling
Create a dataset where one feature has huge scale and show accuracy drops. Then fix with standardization.


In [ ]:
Xtr, ytr, Xte, yte = make_blobs(d=2)
# blow up feature 1 scale
Xtr_bad = Xtr.copy(); Xte_bad = Xte.copy()
Xtr_bad[:, 1] *= 1000
Xte_bad[:, 1] *= 1000

yhat_bad = knn_predict(Xtr_bad, ytr, Xte_bad, k=5)
acc_bad = accuracy(yte, yhat_bad)
print('acc_bad', acc_bad)

# TODO: standardize using train mean/std and re-evaluate
mu = ...
sd = ...
Xtr_std = ...
Xte_std = ...

yhat_std = knn_predict(Xtr_std, ytr, Xte_std, k=5)
acc_std = accuracy(yte, yhat_std)
print('acc_std', acc_std)
check('improves', acc_std >= acc_bad)

---
## Submission Checklist
- All TODOs completed
- k-sweep results shown
- concentration experiment done
- scaling fix demonstrated
